# LSTM Predection

In [1]:
# No need to do this in Azure
# Replace this with the actual directory containing the notebook and Python scripts
# %cd stock-pred

In [2]:
# Global settings
WINDOW = 50
EPOCHS = 15
BATCH = 20
TEST_RATIO = .2

DATASET_NAME = 'stock_prices'
DATASET_ROOT = './dataset/'
DATA_PATH = DATASET_ROOT + '/20220512_tesla.pkl'
LOAD_REGISTERED_MODEL = True
TRAINED_LOAD_PATH = './trained_pred/' if LOAD_REGISTERED_MODEL else None
TRAINED_ROOT = (TRAINED_LOAD_PATH + '/trained/') if LOAD_REGISTERED_MODEL else './trained/'
SCALER_PATH = TRAINED_ROOT +  '/scaler.pkl'
MODEL_PATH = TRAINED_ROOT + '/model'
MODEL_NAME = 'stock-pred-lstm'

# Download dataset
from azureml.core import Dataset, Workspace

ws = Workspace.from_config()
ds = Dataset.get_by_name(ws, name=DATASET_NAME)
# ds_mount = ds.as_mount(path_on_compute='/dataset/')
ds.download(DATASET_ROOT, overwrite=True)
! ls -la ./dataset

# Forces to use CPU rather than GPU
# NVIDIA drivers of higher versions have messy implimentation of LSTM!
# Ref: https://github.com/mozilla/DeepSpeech/issues/3088#issuecomment-656056969
# Ref: https://github.com/tensorflow/tensorflow/issues/35950#issuecomment-577427083
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

total 19
drwxrwxrwx 2 root root     0 Jun 13 10:17 .
drwxrwxrwx 2 root root     0 Jun 13 06:47 ..
-rwxrwxrwx 1 root root   315 Jun 13 10:22 .amlignore
-rwxrwxrwx 1 root root   315 Jun 13 10:22 .amlignore.amltmp
-rwxrwxrwx 1 root root 17926 Jun 14 02:08 20220512_tesla.pkl


## Load Data and Model

In [3]:
from train import (load_data, load_model, load_scaler, extract_x_y)

stock_dates, stock_prices = load_data(DATA_PATH)

if LOAD_REGISTERED_MODEL:
    from azureml.core.model import Model
    model = Model(ws, name=MODEL_NAME)
    model.download(target_dir=TRAINED_ROOT, exist_ok=True)
    
scaler = load_scaler(SCALER_PATH)
scaled_data = scaler.transform(stock_prices[:,None])
train_split = int(len(scaled_data) * (1.0 - TEST_RATIO))
x_test, _ = extract_x_y(scaled_data, window=WINDOW, offset=train_split)

model = load_model(MODEL_PATH)

## Make Prediction

In [4]:
from train import (predict, get_rmse, get_mape)

y_lstm_scaled = model.predict(x_test)
y_lstm = scaler.inverse_transform(y_lstm_scaled)
y_actual = stock_prices[train_split:]

rmse_lstm = get_rmse(pred=y_lstm, actual=y_actual)
mape_lstm = get_mape(pred=y_lstm, actual=y_actual)
print(f'RMSE: {rmse_lstm}, MAPE: {mape_lstm}')

RMSE: 131.68806420557846, MAPE: 0.1210426860978854


In [5]:
import matplotlib.pyplot as plt

def plot_stock(dates, y_pred, y_actual, train_pos):
    fig = plt.figure(figsize=(20, 10))
    plt.plot(dates[train_pos:], y_actual, label='Test Closing Price')
    plt.plot(dates[train_pos:], y_pred, label='Pred Closing Price')
    plt.title('Stock Price Prediction')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend(loc='upper left')

plot_stock(stock_dates, y_lstm, y_actual, train_split)